In [3]:
# necessary packages #

#using Pkg
#Pkg.add("Distances")
using Distributions
using Random
using Distances
using LinearAlgebra
using SparseArrays
using IterativeSolvers
using ProgressMeter
using JLD2

In [4]:
# Generate simulation data #

Random.seed!(123);
N = 1200 # sample size
Nobs = 1000;
q = 10; K = 50; 

sqrt_Σ_diag = sqrt.([0.5, 1, 0.4, 2, 0.3, 2.5, 3.5, 0.45, 1.5, 0.5]);

β = [1.0 -1.0 1.0 -0.5 2.0 -1.5 0.5 0.3 -2.0 1.5
     -5.0 2.0 3.0 -2.0 -6.0 4.0 5.0 -3.0 6.0 -4.0
     8.0 6.9 -12.0 0.0 -4.0 7.7 -8.8 3.3 6.6 -5.5];

ϕ = rand(Uniform(3 / sqrt(2), 30 / sqrt(2)), K);
Λ = rand(Uniform(-2, 2), K, q); # loading matrix

In [5]:
@save "sim3_pars.jld"